<a href="https://colab.research.google.com/github/haeniKim/metaverse-academy/blob/master/NLP/230710_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 29.4 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()
text = '나는 오늘 파이썬을 공부하고 있습니다. 파이썬은 정말재미있는 공부입니다.'

In [ ]:
document = text.replace('.','')
tokenized_document = okt.morphs(document)
tokenized_document

['나',
 '는',
 '오늘',
 '파이썬',
 '을',
 '공부',
 '하고',
 '있습니다',
 '파이썬',
 '은',
 '정말',
 '재미있는',
 '공부',
 '입니다']

In [ ]:
word_to_index = {}
bow = []

for word in tokenized_document:
  if word not in word_to_index.keys():
    word_to_index[word] = len(word_to_index)
    bow.insert(len(word_to_index)-1, 1) #처음 들어가는 단어에는 1의 값을 넣음

  else: #재등장 단어
    index = word_to_index.get(word)
    bow[index] += 1

print(word_to_index)
print(bow)



{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8, '정말': 9, '재미있는': 10, '입니다': 11}
[1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1]


 * DTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = [
    ['누구나','한번쯤','사랑','웃고'],
    ['누구나','한번쯤','사랑','웃고'],
    ['그것','바로','사랑','사랑','사랑']
]

count_vec = CountVectorizer(tokenizer=lambda x:x, lowercase = False)
result = count_vec.fit_transform(text)
print(count_vec.vocabulary_)


{'누구나': 1, '한번쯤': 5, '사랑': 3, '웃고': 4, '그것': 0, '바로': 2}


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


##TF - IDF

In [ ]:
import pandas as pd
from math import log
okt = Okt()

In [ ]:
data = '''
나는 영어를 좋아합니다.
영어, 수학을 매일 공부합니다.
과학 공부를 좋아하고 수학도 좋아 합니다.
영어, 수학, 과학을 좋아하고 영어는 매일 공부 합니다.
'''

In [ ]:
words = okt.morphs(data)

result = []
for temp in words:
  if len(temp) > 1: #한글자 이상인 단어만 리스트에 저장
    result.append(temp)

result

['영어',
 '좋아합니다',
 '영어',
 '수학',
 '매일',
 '공부',
 '합니다',
 '과학',
 '공부',
 '좋아하고',
 '수학',
 '좋아',
 '합니다',
 '영어',
 '수학',
 '과학',
 '좋아하고',
 '영어',
 '매일',
 '공부',
 '합니다']

In [ ]:
result = list(set(result))
result

['좋아하고', '합니다', '공부', '매일', '수학', '좋아합니다', '좋아', '영어', '과학']

In [ ]:
documents = data.split('.')
documents

['\n나는 영어를 좋아합니다',
 '\n영어, 수학을 매일 공부합니다',
 '\n과학 공부를 좋아하고 수학도 좋아 합니다',
 '\n영어, 수학, 과학을 좋아하고 영어는 매일 공부 합니다',
 '\n']

In [ ]:
d_list = []
for temp in documents:
  if temp.split() != '':
    d_list.append(temp.strip())

d_list

['나는 영어를 좋아합니다',
 '영어, 수학을 매일 공부합니다',
 '과학 공부를 좋아하고 수학도 좋아 합니다',
 '영어, 수학, 과학을 좋아하고 영어는 매일 공부 합니다',
 '']

In [ ]:
N = len(d_list)

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in d_list:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d) * idf(t)

#tf('영어','나는 영어를 좋아한다.')

In [ ]:
word_list = result
t_result = []

for temp in range(N):
  t_result.append([])
  d = d_list[temp]

  for temp1 in range(len(word_list)):
    t = word_list[temp1]
    t_result[-1].append(tf(t, d))

print(t_result)

[[0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0, 0, 1, 0], [1, 1, 1, 0, 1, 0, 2, 0, 1], [1, 1, 1, 1, 1, 0, 1, 2, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(t_result , columns = word_list)
df

,좋아하고,합니다,공부,매일,수학,좋아합니다,좋아,영어,과학
0,0,1,0,0,0,1,1,1,0
1,0,1,1,1,1,0,0,1,0
2,1,1,1,0,1,0,2,0,1
3,1,1,1,1,1,0,1,2,1
4,0,0,0,0,0,0,0,0,0


In [ ]:
tfidf_result = []
for temp in range(len(word_list)):
  t = word_list[temp]
  tfidf_result.append(idf(t))

print(tfidf_result) #각 단어들의 가중치

[0.5108256237659907, 0.0, 0.22314355131420976, 0.5108256237659907, 0.22314355131420976, 0.9162907318741551, 0.22314355131420976, 0.22314355131420976, 0.5108256237659907]


In [ ]:
df_idf = pd.DataFrame(tfidf_result, index=word_list, columns=['IDF'])
df_idf

,IDF
좋아하고,0.510826
합니다,0.000000
공부,0.223144
매일,0.510826
수학,0.223144
좋아합니다,0.916291
좋아,0.223144
영어,0.223144
과학,0.510826


In [ ]:
result = []

for temp in range(N):
    result.append([])
    d = d_list[temp]
    for temp in range(len(word_list)):
        t = word_list[temp]
        result[-1].append(tfidf(t,d))

tfidf_df = pd.DataFrame(result,columns=word_list)
tfidf_df


,좋아하고,합니다,공부,매일,수학,좋아합니다,좋아,영어,과학
0,0.000000,0.0,0.000000,0.000000,0.000000,0.916291,0.223144,0.223144,0.000000
1,0.000000,0.0,0.223144,0.510826,0.223144,0.000000,0.000000,0.223144,0.000000
2,0.510826,0.0,0.223144,0.000000,0.223144,0.000000,0.446287,0.000000,0.510826
3,0.510826,0.0,0.223144,0.510826,0.223144,0.000000,0.223144,0.446287,0.510826
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
